<a href="https://colab.research.google.com/github/Mariehf64/fev23cda_french_industry/blob/Dominique_1_Exploration/Exploration_Base_etablissements_par_tranche_effectifs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd  # appel des bibliohthéques
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# appel du fichier source  => base_etablissement_par_tranche_effectif.csv
source=pd.read_csv('/content/base_etablissement_par_tranche_effectif.csv')

# création d'une copie pour éviter tout corruption de data
df=source.copy() 

In [ ]:
# prémière visusalisation 
df.head()

,CODGEO,LIBGEO,REG,DEP,E14TST,E14TS0ND,E14TS1,E14TS6,E14TS10,E14TS20,E14TS50,E14TS100,E14TS200,E14TS500
0,01001,L'Abergement-Clémenciat,82,01,25,22,1,2,0,0,0,0,0,0
1,01002,L'Abergement-de-Varey,82,01,10,9,1,0,0,0,0,0,0,0
2,01004,Ambérieu-en-Bugey,82,01,996,577,272,63,46,24,9,3,2,0
3,01005,Ambérieux-en-Dombes,82,01,99,73,20,3,1,2,0,0,0,0
4,01006,Ambléon,82,01,4,4,0,0,0,0,0,0,0,0


In [ ]:
# premières analyses 

df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36681 entries, 0 to 36680
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   CODGEO    36681 non-null  object
 1   LIBGEO    36681 non-null  object
 2   REG       36681 non-null  int64 
 3   DEP       36681 non-null  object
 4   E14TST    36681 non-null  int64 
 5   E14TS0ND  36681 non-null  int64 
 6   E14TS1    36681 non-null  int64 
 7   E14TS6    36681 non-null  int64 
 8   E14TS10   36681 non-null  int64 
 9   E14TS20   36681 non-null  int64 
 10  E14TS50   36681 non-null  int64 
 11  E14TS100  36681 non-null  int64 
 12  E14TS200  36681 non-null  int64 
 13  E14TS500  36681 non-null  int64 
dtypes: int64(11), object(3)
memory usage: 3.9+ MB


**Concernant les NaN**<br>
aucun NaN

**Concernant les Dtype**<br>
Dtype de la variable DEP doit sans doute être modifié en objet 


In [12]:
#rappport d'exploration

rap=pd.DataFrame() #création du rapport

rap['Colonne']=df.columns  # création de l'index

rename_axis=('N°Col')

rap['Dtype']=[df[i].dtype for i in df.columns ] # création de la colonne dtype

rap['T_NaN']=[(df[i].isna().sum()/df.shape[0]).round(1) for i in df.columns] # création de la colonne taux de NaN 

# création de la colonne max
max=[]
for i in df.columns :
  if df[i].dtype==object:
    max.append('nd')
  else :
    max.append(df[i].max())

rap['Max']=pd.DataFrame(max)

# création del a colonne min
min=[]
for i in df.columns :
  if df[i].dtype==object:
    min.append('nd')
  else :
    min.append(df[i].min())
rap['Min']=pd.DataFrame(min)



display(rap)


,Colonne,Dtype,T_NaN,Max,Min
0,CODGEO,object,0.0,nd,nd
1,LIBGEO,object,0.0,nd,nd
2,REG,int64,0.0,94,1
3,DEP,object,0.0,nd,nd
4,E14TST,int64,0.0,427385,0
5,E14TS0ND,int64,0.0,316603,0
6,E14TS1,int64,0.0,76368,0
7,E14TS6,int64,0.0,14836,0
8,E14TS10,int64,0.0,10829,0
9,E14TS20,int64,0.0,5643,0
